In [47]:
import re

data = """
所学专业 添加
581 经济学关闭 
337 金融学关闭 
305 食品科学关闭 
285 计算机科学关闭 
247 计算科学关闭 
243 生物科学关闭 
217 动物学关闭 
203 工商管理关闭 
202 统计学关闭 
193 生物学关闭 
190 传播学关闭 
162 兽医学关闭 
154 电气与电子工程关闭 
152 数学关闭 
138 会计及相关服务关闭


"""

# 使用正则表达式提取数字和文字
#正则表达式为 (\d+)\s*([\u4e00-\u9fa5/]+)，其中 (\d+) 匹配数字，\s+ 匹配一个或多个空格， [\u4e00-\u9fa5] 匹配中文字符，([a-zA-Z/]+) 匹配文字
#matches = re.findall(r'(\d+)\s+([a-zA-Z/]+)', data)
#matches = re.findall(r'(\d+)\s*([\u4e00-\u9fa5/]+)', data)

#修改了匹配数字的部分，允许匹配逗号分隔的数字。
matches = re.findall(r'(\d{1,3}(?:,\d{3})*)\s+([\u4e00-\u9fa5a-zA-Z\s&()/-]+)', data)

# 打印结果
for match in matches:
    number, text = match
    text = text.replace("打开", "").replace("开启", "").replace("关闭", "")
    print("{} {}".format(text.strip(), number))  

经济学 581
金融学 337
食品科学 305
计算机科学 285
计算科学 247
生物科学 243
动物学 217
工商管理 203
统计学 202
生物学 193
传播学 190
兽医学 162
电气与电子工程 154
数学 152
会计及相关服务 138


合并备忘：

整体：
计算机科学，计算科学 -> 计算机科学
食品科学与工程，食品科学 -> 食品科学与工程
生物科学，生物技术 -> 生物科学

中国：
兽医技术员与兽医助理，兽医学：兽医学
食品科学与工程，食品科学：食品科学与工程

海外：
计算机科学，计算科学：计算机科学
生物科学，生物学：生物科学
会计及相关服务，会计学：会计学

美加：
计算机科学，计算科学：计算机科学
生物科学，生物学：生物科学

英荷：
会计学，会计与财务，会计及相关服务：会计学
金融学，食品科学与工程，生物科学，计算机科学

美东：
计算机科学，计算科学：计算机科学
食品科学与工程

美西：
计算机科学，计算科学：计算机科学
生物科学，生物学，生物技术：生物学
食品科学与工程

澳：
会计及相关服务 ，会计学：会计学
计算机科学 ，计算科学，信息技术  ：计算机科学 
食品科学 ，食品科学与工程 ：食品科学与工程

日新：
食品科学 ，食品科学与工程 ：食品科学与工程 
信息学/信息研究 ，计算机科学 ：计算机科学

In [52]:
import pandas as pd
from io import StringIO

# 输入数据
data_str = """
专业	专业人数
经济学	581
金融学	337
食品科学	305
计算机科学	285
计算科学	247
生物科学	243
动物学	217
工商管理	203
统计学	202
生物学	193
传播学	190
兽医学	162
电气与电子工程	154
数学	152
会计及相关服务	138
"""

# 将字符串转为DataFrame，去除专业人数中的逗号
#df = pd.read_csv(StringIO(data_str), sep='\t')
df = pd.read_csv(StringIO(data_str.replace(',', '')), sep='\t')


# 合并"食品科学"，"食品科学与工程"为"食品科学与工程"
# 合并"国际金融"，"银行与金融支持服务"，"金融学"为"金融学"
# 合并"信息技术"，"信息学/信息研究"，"计算机科学"为"计算机科学" 

# 将"会计与财务"替换为"会计学"
# 将"食品科学”替换为“食品科学与工程”
# 将"生物科学"替换为"生物学"


df['专业'] = df['专业'].replace({'食品科学': '食品科学与工程'})
df['专业'] = df['专业'].replace({'国际金融': '金融学', '银行与金融支持服务': '金融学'})
df['专业'] = df['专业'].replace({'会计与财务': '会计学'})


df['专业'] = df['专业'].replace({'生物科学': '生物学', '生物技术':'生物学'})
#生物化学与分子生物学, 生物统计学
df['专业'] = df['专业'].replace({'信息学/信息研究':'计算机科学', '计算科学':'计算机科学'})

df['专业'] = df['专业'].replace({'兽医技术员与兽医助理': '兽医学'})

df['专业'] = df['专业'].replace({'会计及相关服务': '会计学'})


df = df.groupby('专业', as_index=False).sum()

# 按专业人数从大到小排序
df = df.sort_values(by='专业人数', ascending=False)

# 打印结果
print(df)

         专业  专业人数
8       经济学   581
10    计算机科学   532
6       生物学   436
11      金融学   337
12  食品科学与工程   305
3       动物学   217
4      工商管理   203
9       统计学   202
1       传播学   190
2       兽医学   162
7   电气与电子工程   154
5        数学   152
0       会计学   138
